In [1]:
!pip install pandas
!pip install ortools
from ortools.linear_solver import pywraplp

# divide points by price points/price - linear prgram finds the best value players and may choose the player with lless points but higher value 

In [30]:
import pandas as pd

from google.colab import files

df = pd.read_csv('fantasy_football.csv')
print (df)

                    Name Position Team  ProjectedPoints  Price
0          Aaron Rodgers       QB  GBP            333.5   51.2
1              Tom Brady       QB  NEP            312.1   38.3
2             Drew Brees       QB  NOS            306.2   36.3
3              Matt Ryan       QB  ATL            294.3   29.2
4         Russell Wilson       QB  SEA            287.9   25.8
..                   ...      ...  ...              ...    ...
947        Chicago Bears      DEF  CHI             90.2    1.0
948  San Francisco 49ers      DEF  SFO             89.3    1.0
949   New Orleans Saints      DEF  NOS             89.2    1.0
950        New York Jets      DEF  NYJ             87.0    1.0
951     Cleveland Browns      DEF  CLE             86.3    1.0

[952 rows x 5 columns]


In [31]:
solver = pywraplp.Solver.CreateSolver("SCIP")

In [32]:
# create dataframesfor each position 
QB = df.loc[df.Position=='QB'].reset_index()
WR = df.loc[df.Position=='WR'].reset_index()
RB = df.loc[df.Position=='RB'].reset_index()
TE = df.loc[df.Position=='TE'].reset_index()
K = df.loc[df.Position=='K'].reset_index()
DEF = df.loc[df.Position=='DEF'] .reset_index()

In [33]:
QB_dummy = [[],[]]
WR_dummy = [[],[],[],[]]
RB_dummy = [[],[],[]]
TE_dummy = [[],[]]
K_dummy = [[],[]]
DEF_dummy = [[],[]]

In [34]:
for index,row in df.iterrows():
  if row['Position'] == 'QB':
    for i in range(len(QB_dummy)):
      QB_dummy[i].append(solver.IntVar(0,1, row['Name'] + str(i)))
  if row['Position'] == 'WR':
    for i in range(len(WR_dummy)):
      WR_dummy[i].append(solver.IntVar(0,1, row['Name'] + str(i)))
  if row['Position'] == 'RB':
    for i in range(len(RB_dummy)):
      RB_dummy[i].append(solver.IntVar(0,1, row['Name'] + str(i)))
  if row['Position'] == 'TE':
    for i in range(len(TE_dummy)):
      TE_dummy[i].append(solver.IntVar(0,1, row['Name'] + str(i)))
  if row['Position'] == 'K':
    for i in range(len(K_dummy)):
      K_dummy[i].append(solver.IntVar(0,1, row['Name'] + str(i)))
  if row['Position'] == 'DEF':
    for i in range(len(DEF_dummy)):
      DEF_dummy[i].append(solver.IntVar(0,1, row['Name'] + str(i)))

      


In [35]:
# what starting roser is going to give us the most points 
solver.Maximize(sum(QB_dummy[0][i]*QB.at[i, 'ProjectedPoints'] for i in range(len(QB)))+
                sum(WR_dummy[0][i]*WR.at[i, 'ProjectedPoints'] for i in range(len(WR)))+
                sum(RB_dummy[0][i]*RB.at[i, 'ProjectedPoints'] for i in range(len(RB)))+
                sum(TE_dummy[0][i]*TE.at[i, 'ProjectedPoints'] for i in range(len(TE)))+
                sum(K_dummy[0][i]*K.at[i, 'ProjectedPoints'] for i in range(len(K)))+
                sum(DEF_dummy[0][i]*DEF.at[i, 'ProjectedPoints'] for i in range(len(DEF)))+
                sum(WR_dummy[1][i]*WR.at[i, 'ProjectedPoints'] for i in range(len(WR)))+
                sum(WR_dummy[2][i]*WR.at[i, 'ProjectedPoints'] for i in range(len(WR)))+
                sum(RB_dummy[1][i]*RB.at[i, 'ProjectedPoints'] for i in range(len(RB)))+
                sum(.5*QB_dummy[1][i]*QB.at[i, 'ProjectedPoints'] for i in range(len(QB)))+
                sum(.5*WR_dummy[3][i]*WR.at[i, 'ProjectedPoints'] for i in range(len(WR)))+
                sum(.5*RB_dummy[2][i]*RB.at[i, 'ProjectedPoints'] for i in range(len(RB)))+
                sum(.5*TE_dummy[1][i]*TE.at[i, 'ProjectedPoints'] for i in range(len(TE)))+
                sum(.5*K_dummy[1][i]*K.at[i, 'ProjectedPoints'] for i in range(len(K)))+
                sum(.5*DEF_dummy[1][i]*DEF.at[i, 'ProjectedPoints'] for i in range(len(DEF))))



In [36]:
# do this same thing for TE, K, DEF 
solver.Add(sum(QB_dummy[0][i] for i in range(len(QB_dummy[0])))+sum(QB_dummy[1][i] for i in range(len(QB_dummy[1])))<=2)#max number of qbs allowed on roster.
solver.Add(sum(TE_dummy[0][i] for i in range(len(TE_dummy[0])))+sum(TE_dummy[1][i] for i in range(len(TE_dummy[1])))<=2)

solver.Add(sum(K_dummy[0][i] for i in range(len(K_dummy[0])))+sum(K_dummy[1][i] for i in range(len(K_dummy[1])))<=1)
solver.Add(sum(DEF_dummy[0][i] for i in range(len(DEF_dummy[0])))+sum(DEF_dummy[1][i] for i in range(len(DEF_dummy[1])))<=1)

#for wr sum up all four dummy lksts in wr= <4
solver.Add(sum(WR_dummy[0][i] for i in range(len(WR_dummy[0])))+
               sum(WR_dummy[1][i] for i in range(len(WR_dummy[1])))+
                   sum(WR_dummy[2][i] for i in range(len(WR_dummy[2])))+
                       sum(WR_dummy[3][i] for i in range(len(WR_dummy[3])))<=8)

#for rb sum up all 3 dummy lksts <= 3
solver.Add(sum(RB_dummy[0][i] for i in range(len(RB_dummy[0])))+
               sum(RB_dummy[1][i] for i in range(len(RB_dummy[1])))+
                  sum(RB_dummy[2][i] for i in range(len(RB_dummy[2])))<=8)


#Must have a starting QB - for all of the ones that do not hav a .5 
solver.Add(sum(QB_dummy[0][i] for i in range(len(QB_dummy[0])))==1)

solver.Add(sum(WR_dummy[0][i] for i in range(len(WR_dummy[0])))==1)
solver.Add(sum(WR_dummy[1][i] for i in range(len(WR_dummy[1])))==1)
solver.Add(sum(WR_dummy[2][i] for i in range(len(WR_dummy[2])))==1)

solver.Add(sum(RB_dummy[0][i] for i in range(len(RB_dummy[0])))==1)
solver.Add(sum(RB_dummy[1][i] for i in range(len(RB_dummy[1])))==1)       

solver.Add(sum(TE_dummy[0][i] for i in range(len(TE_dummy[0])))==1)
solver.Add(sum(TE_dummy[1][i] for i in range(len(TE_dummy[0])))==1)

solver.Add(sum(K_dummy[0][i] for i in range(len(K_dummy[0])))==1)
solver.Add(sum(DEF_dummy[0][i] for i in range(len(DEF_dummy[0])))==1)


<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fac8052bea0> >

In [46]:
solver.Add(sum(QB_dummy[0][i]*QB.at[i, 'Price'] for i in range(len(QB)))+
                sum(RB_dummy[0][i]*RB.at[i, 'Price'] for i in range(len(RB)))+
                sum(RB_dummy[1][i]*RB.at[i, 'Price'] for i in range(len(RB)))+
                
                sum(WR_dummy[0][i]*WR.at[i, 'Price'] for i in range(len(WR)))+             
                sum(WR_dummy[1][i]*WR.at[i, 'Price'] for i in range(len(WR)))+
                sum(WR_dummy[2][i]*WR.at[i, 'Price'] for i in range(len(WR)))+
           
                sum(TE_dummy[0][i]*TE.at[i, 'Price'] for i in range(len(TE)))+
                sum(K_dummy[0][i]*K.at[i, 'Price'] for i in range(len(K)))+
                sum(DEF_dummy[0][i]*DEF.at[i, 'Price'] for i in range(len(DEF)))+
                
                              
                sum(QB_dummy[1][i]*QB.at[i, 'Price'] for i in range(len(QB)))+
                sum(WR_dummy[3][i]*WR.at[i, 'Price'] for i in range(len(WR)))+
                sum(RB_dummy[2][i]*RB.at[i, 'Price'] for i in range(len(RB)))+
                sum(TE_dummy[1][i]*TE.at[i, 'Price'] for i in range(len(TE)))+
                #sum(K_dummy[1][i]*K.at[i, 'Price'] for i in range(len(K)))+
                sum(DEF_dummy[1][i]*DEF.at[i, 'Price'] for i in range(len(DEF)))<=200)


<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fac8014c300> >

In [47]:
# SUM OF ALL PPL <= 15 
solver.Add(sum(QB_dummy[0][i] for i in range(len(QB_dummy[0])))+sum(QB_dummy[1][i] for i in range(len(QB_dummy[1])))+
           sum(TE_dummy[0][i] for i in range(len(TE_dummy[0])))+sum(TE_dummy[1][i] for i in range(len(TE_dummy[1])))+
           sum(K_dummy[0][i] for i in range(len(K_dummy[0])))+sum(K_dummy[1][i] for i in range(len(K_dummy[1])))+
           sum(DEF_dummy[0][i] for i in range(len(DEF_dummy[0])))+sum(DEF_dummy[1][i] for i in range(len(DEF_dummy[1])))+
           sum(WR_dummy[0][i] for i in range(len(WR_dummy[0])))+
            sum(WR_dummy[1][i] for i in range(len(WR_dummy[1])))+
            sum(WR_dummy[2][i] for i in range(len(WR_dummy[2])))+
            sum(WR_dummy[3][i] for i in range(len(WR_dummy[3])))+
           sum(RB_dummy[0][i] for i in range(len(RB_dummy[0])))+
               sum(RB_dummy[1][i] for i in range(len(RB_dummy[1])))+
                  sum(RB_dummy[2][i] for i in range(len(RB_dummy[2])))<=15)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fac801dd330> >

In [48]:
# these constraints make sure proper number 
QB_dummy[0][0], QB_dummy[0][1]
QB_dummy[0][1], QB_dummy[0][1]

for i in range(len(QB)):
  solver.Add(QB_dummy[0][i] + QB_dummy[1][i] <=1)
for i in range(len(TE)):
  solver.Add(TE_dummy[0][i] + TE_dummy[1][i] <=1)
for i in range(len(K)):
  solver.Add(K_dummy[0][i] + K_dummy[1][i] <=1) 
for i in range(len(DEF)):
  solver.Add(DEF_dummy[0][i] + DEF_dummy[1][i] <=1) 

# WR 
for i in range(len(WR)):
  solver.Add(WR_dummy[0][i] + WR_dummy[1][i] + WR_dummy[2][i]+WR_dummy[3][i]<=1)
             
#RB
for i in range(len(RB)):
  solver.Add(RB_dummy[0][i] + RB_dummy[1][i] + RB_dummy[2][i]<=1) 


In [59]:
status = solver.Solve()

priceCount = 0

if status == pywraplp.Solver.OPTIMAL:
  for col in range(len(QB_dummy)): # column 
    for plDummy in range(len(QB)):
      if QB_dummy[col][plDummy].solution_value() > 0: 
        if col == 0:
          print("Starter QB: ", QB_dummy[col][plDummy], "Fantasy_Points:",QB.at[plDummy, 'ProjectedPoints'],"Price: ", QB.at[plDummy, "Price"])
        else:
          priceCount += QB.at[plDummy, "Price"]
          print("Backup QB: ", QB_dummy[col][plDummy], "Fantasy_Points:",QB.at[plDummy, 'ProjectedPoints'],"Price: ", QB.at[plDummy, "Price"])

  for col in range(len(TE_dummy)): # column 
    for plDummy in range(len(TE)):
      if TE_dummy[col][plDummy].solution_value() > 0: 
        if col == 0:
          priceCount += QB.at[plDummy, "Price"]
          print("Starter TE: ", TE_dummy[col][plDummy], "Fantasy_Points:",TE.at[plDummy, 'ProjectedPoints'],"Price: ", TE.at[plDummy, "Price"])
        else:
          print("Backup TE: ", TE_dummy[col][plDummy], "Fantasy_Points:",TE.at[plDummy, 'ProjectedPoints'],"Price: ", TE.at[plDummy, "Price"])
  for col in range(len(K_dummy)): # column 
    for plDummy in range(len(K)):
      if K_dummy[col][plDummy].solution_value() > 0: 
        if col == 0:
          priceCount += QB.at[plDummy, "Price"]
          print("Starter K: ", K_dummy[col][plDummy], "Fantasy_Points:",K.at[plDummy, 'ProjectedPoints'],"Price: ", K.at[plDummy, "Price"])
        else:
          priceCount += QB.at[plDummy, "Price"]
          print("Backup K: ", K_dummy[col][plDummy], "Fantasy_Points:",K.at[plDummy, 'ProjectedPoints'],"Price: ", K.at[plDummy, "Price"])
  for col in range(len(DEF_dummy)): # column 
    for plDummy in range(len(DEF)):
      if DEF_dummy[col][plDummy].solution_value() > 0: 
        if col == 0:
          priceCount += QB.at[plDummy, "Price"]
          print("Starter DEF: ", DEF_dummy[col][plDummy], "Fantasy_Points:",DEF.at[plDummy, 'ProjectedPoints'],"Price: ", DEF.at[plDummy, "Price"])
        else:
          priceCount += QB.at[plDummy, "Price"]
          print("Backup DEF: ", DEF_dummy[col][plDummy], "Fantasy_Points:",DEF.at[plDummy, 'ProjectedPoints'],"Price: ", DEF.at[plDummy, "Price"])

  for col in range(len(WR_dummy)): # column 
    for plDummy in range(len(WR)):
      if WR_dummy[col][plDummy].solution_value() > 0: 
        if col > 2:
          priceCount += QB.at[plDummy, "Price"]
          print("Backup WR: ", WR_dummy[col][plDummy], "Fantasy_Points:",WR.at[plDummy, 'ProjectedPoints'],"Price: ", WR.at[plDummy, "Price"])
        else:
          priceCount += QB.at[plDummy, "Price"]
          print("Starter WR: ", WR_dummy[col][plDummy], "Fantasy_Points:",WR.at[plDummy, 'ProjectedPoints'],"Price: ", WR.at[plDummy, "Price"])
  
  for col in range(len(RB_dummy)): # column 
    for plDummy in range(len(RB)):
      if RB_dummy[col][plDummy].solution_value() > 0: 
        if col > 1:
          priceCount += QB.at[plDummy, "Price"]
          print("Backup RB: ", RB_dummy[col][plDummy], "Fantasy_Points:",RB.at[plDummy, 'ProjectedPoints'],"Price: ", RB.at[plDummy, "Price"])
        else:
          priceCount += QB.at[plDummy, "Price"]
          print("Starter RB: ", RB_dummy[col][plDummy], "Fantasy_Points:",RB.at[plDummy, 'ProjectedPoints'],"Price: ", RB.at[plDummy, "Price"])

  print("Maximum Fantasy Score: ",solver.Objective().Value())

print("price count: ", priceCount)

Starter QB:  Cam Newton0 Fantasy_Points: 278.8 Price:  16.6
Backup QB:  Tyrod Taylor1 Fantasy_Points: 255.8 Price:  5.1
Starter TE:  Rob Gronkowski0 Fantasy_Points: 146.0 Price:  35.9
Backup TE:  Coby Fleener1 Fantasy_Points: 81.7 Price:  1.6
Starter K:  Justin Tucker0 Fantasy_Points: 147.0 Price:  9.2
Starter DEF:  Denver Broncos0 Fantasy_Points: 129.0 Price:  7.6
Starter WR:  Michael Crabtree0 Fantasy_Points: 130.9 Price:  18.3
Starter WR:  Julio Jones1 Fantasy_Points: 187.8 Price:  51.3
Starter WR:  Emmanuel Sanders2 Fantasy_Points: 125.7 Price:  15.2
Starter RB:  C.J. Anderson0 Fantasy_Points: 128.8 Price:  7.9
Starter RB:  Ty Montgomery1 Fantasy_Points: 140.9 Price:  14.9
Backup RB:  Terrance West2 Fantasy_Points: 120.1 Price:  5.5
Backup RB:  Spencer Ware2 Fantasy_Points: 115.5 Price:  4.3
Backup RB:  Matt Forte2 Fantasy_Points: 112.2 Price:  3.5
Backup RB:  Jonathan Stewart2 Fantasy_Points: 111.7 Price:  2.9
Maximum Fantasy Score:  1813.4
price count:  195.3
